# Data Exercise

This data exercise represents an example of the type of data work we complete. We estimate that the exercise will take 2-3 hours to complete. Please use whatever statistical programming language, programming language, or data manipulation tool you are most comfortable with (SAS, R, SPSS, STATA, Python, SQL, etc).Please note, if you choose to alias tables in your code, we ask that you use descriptive aliases and not single letters.

This exercise will evaluate your ability to build a cohort of patients and calculate some metrics related to that cohort. You should have the following:

    5 datasets (all data you need is found within the provided datasets)
    A data dictionary defining each dataset and its fields

If you would like to use a database for this exercise, use these instructions to set up a local Postgres database. Otherwise, you can load the datasets with your programming language of choice.

# Instructions
## Part 1: Assemble the project cohort

The project goal is to identify patients seen for drug overdose, determine if they had an active opioid at the start of the encounter, and if they had any readmissions for drug overdose.

Your task is to assemble the study cohort by identifying encounters that meet the following criteria:

1. The patient’s visit is an encounter for drug overdose
2. The hospital encounter occurs after July 15, 1999
3. The patient’s age at time of encounter is between 18 and 35 (Patient is considered to be 35 until turning 36)

## Part 2: Create additional fields

With your drug overdose encounter, create the following indicators:

    DEATH_AT_VISIT_IND: 1 if patient died during the drug overdose encounter, 0 if the patient died at a different time
    COUNT_CURRENT_MEDS: Count of active medications at the start of the drug overdose encounter
    CURRENT_OPIOID_IND: 1 if the patient had at least one active medication at the start of the overdose encounter that is on the Opioids List (provided below), 0 if not
    READMISSION_90_DAY_IND: 1 if the visit resulted in a subsequent drug overdose readmission within 90 days, 0 if not
    READMISSION_30_DAY_IND: 1 if the visit resulted in a subsequent drug overdose readmission within 30 days, 0 if not overdose encounter, 0 if not
    FIRST_READMISSION_DATE: The date of the index visit's first readmission for drug overdose. Field should be left as N/A if no readmission for drug overdose within 90 days

Part 3: Export the data to a CSV file

Export a dataset containing these required fields:
Field name 	Field Description 	Data Type
PATIENT_ID 	Patient identifier 	Character String
ENCOUNTER_ID 	Visit identifier 	Character string
HOSPITAL_ENCOUNTER_DATE 	Beginning of hospital encounter date 	Date/time
AGE_AT_VISIT 	Patient age at admission 	Num
DEATH_AT_VISIT_IND 	Indicator if the patient died during the drug overdose encounter. Leave N/A if patient has not died, 	0 /1
COUNT_CURRENT_MEDS 	Count of active medications at the start of the drug overdose encounter 	Num
CURRENT_OPIOID_IND 	if the patient had at least one active medication at the start of the overdose encounter that is on the Opioids List (provided below) 	0/1
READMISSION_90_DAY_IND 	Indicator if the visit resulted in a subsequent readmission within 90 days 	0/1
READMISSION_30_DAY_IND 	Indicator if the visit resulted in a subsequent readmission within 30 days 	0/1
FIRST_READMISSION_DATE 	Date of the first readmission for drug overdose within 90 days. Leave N/A if no readmissions for drug overdose within 90 days. 	Date/time
Opioids List:

    Hydromorphone 325Mg
    Fentanyl – 100 MCG
    Oxycodone-acetaminophen 100 Ml

Submission Guidelines

Upon completion, please email the following to DataRecruiting@email.chop.edu:

    Data Exercise output dataset (.csv) (Please name the .csv file in the following format: "FIRSTNAME_LASTNAME.csv")
    Data Exercise code (text file)

Good luck!

In [1]:
import numpy as np
import pandas as pd
import math
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 500)

### import files

#### allergies

In [3]:
allergies_df = pd.read_csv(r'C:\Users\katec\Documents\CHOP\analyst-take-home-task-master\analyst-take-home-task-master\datasets\allergies.csv')

In [4]:
allergies_df.head()

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
0,1983-07-12,NaN,7341aae3-7606-456e-b3c1-f67d1bc6b19b,8ea228c8-f853-431e-969b-c7085602d2c1,300913006,Shellfish allergy
1,2016-10-22,NaN,54e3e6c7-2089-4158-83ef-a1d549bf3523,f70cb7eb-6986-486c-9b43-9377c549b396,419474003,Allergy to mould
2,2016-10-22,NaN,54e3e6c7-2089-4158-83ef-a1d549bf3523,f70cb7eb-6986-486c-9b43-9377c549b396,232350006,House dust mite allergy
3,2016-10-22,NaN,54e3e6c7-2089-4158-83ef-a1d549bf3523,f70cb7eb-6986-486c-9b43-9377c549b396,232347008,Dander (animal) allergy
4,2016-10-22,NaN,54e3e6c7-2089-4158-83ef-a1d549bf3523,f70cb7eb-6986-486c-9b43-9377c549b396,418689008,Allergy to grass pollen


In [5]:
allergies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5374 entries, 0 to 5373
Data columns (total 6 columns):
START          5374 non-null object
STOP           506 non-null object
PATIENT        5374 non-null object
ENCOUNTER      5374 non-null object
CODE           5374 non-null int64
DESCRIPTION    5374 non-null object
dtypes: int64(1), object(5)
memory usage: 252.0+ KB


In [6]:
#drop unnecessary columns for final cohort
allergies_df = allergies_df.drop(['CODE'], axis=1)

#### encounters

In [7]:
encounters_df = pd.read_csv(r'C:\Users\katec\Documents\CHOP\analyst-take-home-task-master\analyst-take-home-task-master\datasets\encounters.csv')

In [8]:
encounters_df.head()

,Id,START,STOP,PATIENT,PROVIDER,ENCOUNTERCLASS,CODE,DESCRIPTION,COST,REASONCODE,REASONDESCRIPTION
0,2590963f-222b-4870-b8c7-86d07f4d44b3,1959-06-18 09:43:49,1959-06-25 21:37:31,3d8e57b2-3de5-4fb6-95b4-8370270c5cc2,fa6006c2-c562-3278-859b-96b266fc3ea1,ambulatory,424441002.0,Prenatal initial visit,105.37,72892002.0,Normal pregnancy
1,ba6137e3-c8cd-42e6-9a02-eafa13d1cfa9,2008-08-29 19:14:19,2008-09-07 13:33:07,553b00b2-347c-48ec-90ee-f59924ff50eb,fe803a6f-c717-36e8-a338-9b3cdb9ee724,outpatient,698314001.0,Consultation for treatment,105.37,NaN,NaN
2,10aa7678-5c84-4ce9-81af-09272307c1f0,1999-07-12 03:05:41,1999-07-20 11:18:24,7f4ea9fb-f436-411e-ab34-e94750edfa93,cc6a930c-727f-3999-b655-179e57b30538,wellness,185349003.0,Encounter for check up (procedure),105.37,NaN,NaN
3,3710b40a-bcc0-4c0f-a204-e47a5f3aec98,1997-09-20 16:09:36,1997-09-30 22:43:32,2043c57c-1085-45a3-87e8-a223d5b6693d,d2c4dfee-3c37-318f-a415-367315d28b65,wellness,185349003.0,Encounter for check up (procedure),105.37,NaN,NaN
4,92078752-a89c-4931-b95e-0f8df485e9f6,1963-09-26 09:43:49,1963-10-05 03:20:42,3d8e57b2-3de5-4fb6-95b4-8370270c5cc2,fa6006c2-c562-3278-859b-96b266fc3ea1,ambulatory,424441002.0,Prenatal initial visit,105.37,72892002.0,Normal pregnancy


In [9]:
encounters_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413298 entries, 0 to 413297
Data columns (total 11 columns):
Id                   413298 non-null object
START                413298 non-null object
STOP                 413298 non-null object
PATIENT              413298 non-null object
PROVIDER             413296 non-null object
ENCOUNTERCLASS       413298 non-null object
CODE                 413297 non-null float64
DESCRIPTION          413298 non-null object
COST                 413298 non-null float64
REASONCODE           134681 non-null float64
REASONDESCRIPTION    134681 non-null object
dtypes: float64(3), object(8)
memory usage: 34.7+ MB


In [10]:
encounters_df['ENCOUNTERCLASS'].unique()

array(['ambulatory', 'outpatient', 'wellness', 'inpatient', 'emergency',
       'urgentcare'], dtype=object)

##### comment/assumptions:

Possible overdose:
encounters_df['ENCOUNTERCLASS'] = 'inpatient', 'emergency';

Unknown if a patient admission through emergency is <23< hrs or is considered an admission for the purpose of this cohort; **WILL INCLUDE** 'emergency' admission if listed as drug overdose.

In [11]:
encounters_df['DESCRIPTION'].unique()

array(['Prenatal initial visit', 'Consultation for treatment',
       'Encounter for check up (procedure)', 'Encounter for symptom',
       'Encounter for problem', 'Patient encounter procedure',
       'Hypertension follow-up encounter',
       'Periodic reevaluation and management of healthy individual (procedure)',
       'Emergency room admission (procedure)', "Encounter for 'check-up'",
       'Prenatal visit', 'Postoperative follow-up visit (procedure)',
       'Screening surveillance (regime/therapy)',
       'Gynecology service (qualifier value)',
       'Domiciliary or rest home patient evaluation and management',
       'Encounter Inpatient', 'Emergency Room Admission',
       'Urgent care clinic (procedure)',
       'Encounter for problem (procedure)',
       'Drug rehabilitation and detoxification',
       'Obstetric emergency hospital admission', 'Postnatal visit',
       'Office Visit', 'Follow-up encounter', 'Encounter for Problem',
       'Asthma follow-up', 'Inpatient 

In [12]:
encounters_df[encounters_df['DESCRIPTION'].str.contains('Drug overdose', na=False)]

,Id,START,STOP,PATIENT,PROVIDER,ENCOUNTERCLASS,CODE,DESCRIPTION,COST,REASONCODE,REASONDESCRIPTION


##### comment/assumptions:

No specific description for 'drug overdose' noted in DESCRIPTION. Will consider the below as possible drug overdose if no definitive description of encounter found.

Possible overdose:
encounters_df['DESCRIPTION'] = 'Consultation for treatment', 'Encounter for problem', 'Screening surveillance (regime/therapy)', 'Encounter Inpatient', 'Emergency Room Admission', 'Encounter for problem (procedure)', 'Drug rehabilitation and detoxification', 'Follow-up encounter', 'Encounter for Problem', 'Inpatient stay (finding)', 'Emergency Encounter', 'Inpatient stay 3 days', 'Death Certification'; 

In [13]:
encounters_df['REASONDESCRIPTION'].unique()

array(['Normal pregnancy', nan, 'Seasonal allergic rhinitis',
       'Acute bronchitis (disorder)', 'Viral sinusitis (disorder)',
       'Concussion with no loss of consciousness', 'Bullet wound',
       'Malignant neoplasm of breast (disorder)',
       'Osteoarthritis of knee', 'Fracture of forearm',
       'Acute viral pharyngitis (disorder)', 'Second degree burn',
       'Sinusitis (disorder)', 'Anemia (disorder)', 'Chronic pain',
       'Fracture of clavicle', 'Otitis media', 'Appendicitis',
       'Child attention deficit disorder',
       'Chronic intractable migraine without aura', 'Impacted molars',
       'Acute bacterial sinusitis (disorder)',
       'Concussion with loss of consciousness', 'Drug overdose',
       'Fracture of rib', 'Malignant tumor of colon',
       "Alzheimer's disease (disorder)", 'Hyperlipidemia', 'Cystitis',
       'Polyp of colon', 'Overlapping malignant neoplasm of colon',
       'Fetus with unknown complication',
       'Escherichia coli urinary tract

In [14]:
encounters_df[encounters_df['REASONDESCRIPTION'].str.contains('Drug overdose', na=False)]

,Id,START,STOP,PATIENT,PROVIDER,ENCOUNTERCLASS,CODE,DESCRIPTION,COST,REASONCODE,REASONDESCRIPTION
355,2a917920-2701-49f0-9340-827320eef76d,2003-03-31 21:50:51,2003-04-08 13:20:43,708b81c9-21a9-411a-aae4-052ef8925b97,fb37c581-84a6-3513-9b1e-1d55e67e887a,emergency,50849002.0,Emergency Room Admission,105.37,55680006.0,Drug overdose
373,22874b3d-0873-40e4-a455-897d51d593f7,2012-02-18 21:50:51,2012-02-28 21:12:17,708b81c9-21a9-411a-aae4-052ef8925b97,fb37c581-84a6-3513-9b1e-1d55e67e887a,emergency,50849002.0,Emergency Room Admission,105.37,55680006.0,Drug overdose
377,134c5ee3-1b72-4e3e-a44a-3efebe77603b,2013-08-03 21:50:51,2013-08-13 07:44:52,708b81c9-21a9-411a-aae4-052ef8925b97,fb37c581-84a6-3513-9b1e-1d55e67e887a,emergency,50849002.0,Emergency Room Admission,105.37,55680006.0,Drug overdose
381,6125f147-72d4-48a4-9813-2a9dc878f055,2014-12-08 21:50:51,2014-12-17 12:25:27,708b81c9-21a9-411a-aae4-052ef8925b97,fb37c581-84a6-3513-9b1e-1d55e67e887a,emergency,50849002.0,Emergency Room Admission,105.37,55680006.0,Drug overdose
390,f837dcf8-af7d-43b8-b5c2-773d494617ab,2015-08-31 21:50:51,2015-09-08 12:04:08,708b81c9-21a9-411a-aae4-052ef8925b97,fb37c581-84a6-3513-9b1e-1d55e67e887a,emergency,50849002.0,Emergency Room Admission,105.37,55680006.0,Drug overdose
1424,2b235f52-2da8-4d48-b100-dfe72e698771,1997-07-06 11:26:13,1997-07-15 12:25:23,722270fe-f8dd-46e6-aace-756450258911,6e099510-4f48-3174-a90c-8f0a832d2c87,emergency,50849002.0,Emergency Room Admission,105.37,55680006.0,Drug overdose
1437,068d0cbf-1625-48ae-9b4c-c32c208489f1,2007-03-17 11:26:13,2007-03-22 20:51:44,722270fe-f8dd-46e6-aace-756450258911,a8f03f32-4618-3503-9f5e-f270d3c116c1,emergency,50849002.0,Emergency Room Admission,105.37,55680006.0,Drug overdose
1438,cfa60809-f10a-47d0-83fd-87f964989490,2007-07-07 11:26:13,2007-07-14 10:18:31,722270fe-f8dd-46e6-aace-756450258911,6e099510-4f48-3174-a90c-8f0a832d2c87,emergency,50849002.0,Emergency Room Admission,105.37,55680006.0,Drug overdose
1475,7b843d12-bfe0-44a8-aa83-3cd98d7344b5,2012-06-07 11:26:13,2012-06-18 01:04:13,722270fe-f8dd-46e6-aace-756450258911,6e099510-4f48-3174-a90c-8f0a832d2c87,emergency,50849002.0,Emergency Room Admission,105.37,55680006.0,Drug overdose
1477,94d455e3-6cdc-4bc6-9555-5e05f817d4da,2012-09-20 11:26:13,2012-09-30 22:39:40,722270fe-f8dd-46e6-aace-756450258911,6e099510-4f48-3174-a90c-8f0a832d2c87,emergency,50849002.0,Emergency Room Admission,105.37,55680006.0,Drug overdose


##### comment/assumptions:

Possible overdose:
encounters_df['REASONDESCRIPTION']= 'Chronic pain', 'Chronic intractable migraine without aura', **'Drug overdose'**, 'Suicidal deliberate poisoning';

**Noted specific REASONDESCRIPTION for 'Drug overdose'. Will use this value to indicate drug overdose for cohort.** 

In [15]:
encounters_df.dtypes['START']

dtype('O')

In [16]:
#convert START (admission) & STOP (discharge) 
#dates from string to date/time
encounters_df['START']= pd.to_datetime(encounters_df['START'])
encounters_df['STOP']= pd.to_datetime(encounters_df['STOP'])

In [17]:
encounters_df.dtypes['STOP']

dtype('<M8[ns]')

In [18]:
#drop columns unnecessary for final cohort
encounters_df = encounters_df.drop(['PROVIDER', 'DESCRIPTION', 'CODE', 'COST', 'REASONCODE'], axis=1)

In [19]:
encounters_df.head()

,Id,START,STOP,PATIENT,ENCOUNTERCLASS,REASONDESCRIPTION
0,2590963f-222b-4870-b8c7-86d07f4d44b3,1959-06-18 09:43:49,1959-06-25 21:37:31,3d8e57b2-3de5-4fb6-95b4-8370270c5cc2,ambulatory,Normal pregnancy
1,ba6137e3-c8cd-42e6-9a02-eafa13d1cfa9,2008-08-29 19:14:19,2008-09-07 13:33:07,553b00b2-347c-48ec-90ee-f59924ff50eb,outpatient,NaN
2,10aa7678-5c84-4ce9-81af-09272307c1f0,1999-07-12 03:05:41,1999-07-20 11:18:24,7f4ea9fb-f436-411e-ab34-e94750edfa93,wellness,NaN
3,3710b40a-bcc0-4c0f-a204-e47a5f3aec98,1997-09-20 16:09:36,1997-09-30 22:43:32,2043c57c-1085-45a3-87e8-a223d5b6693d,wellness,NaN
4,92078752-a89c-4931-b95e-0f8df485e9f6,1963-09-26 09:43:49,1963-10-05 03:20:42,3d8e57b2-3de5-4fb6-95b4-8370270c5cc2,ambulatory,Normal pregnancy


#### medications

In [20]:
medications_df = pd.read_csv(r'C:\Users\katec\Documents\CHOP\analyst-take-home-task-master\analyst-take-home-task-master\datasets\medications.csv')

In [21]:
medications_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112270 entries, 0 to 112269
Data columns (total 11 columns):
START                112270 non-null object
STOP                 92085 non-null object
PATIENT              112270 non-null object
ENCOUNTER            112270 non-null object
CODE                 112270 non-null int64
DESCRIPTION          112270 non-null object
COST                 112270 non-null float64
DISPENSES            112270 non-null int64
TOTALCOST            112270 non-null float64
REASONCODE           72813 non-null float64
REASONDESCRIPTION    75227 non-null object
dtypes: float64(3), int64(2), object(6)
memory usage: 9.4+ MB


In [22]:
medications_df.head()

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,COST,DISPENSES,TOTALCOST,REASONCODE,REASONDESCRIPTION
0,1997-09-20,NaN,2043c57c-1085-45a3-87e8-a223d5b6693d,3710b40a-bcc0-4c0f-a204-e47a5f3aec98,316049,Hydromorphone 325 MG,214.96,265,56964.40,59621000.0,Pain Relief
1,1957-04-19,NaN,50f799aa-740c-4da9-ab85-49d5cbf4046b,64b4efb0-26ba-4c7f-91c1-64d8f62e49e3,1049630,diphenhydrAMINE Hydrochloride 25 MG Oral Tablet,6.53,757,4943.21,NaN,NaN
2,2008-08-29,2009-08-24,553b00b2-347c-48ec-90ee-f59924ff50eb,ba6137e3-c8cd-42e6-9a02-eafa13d1cfa9,749785,Ortho Tri-Cyclen 28 Day Pack,24.12,12,289.44,NaN,NaN
3,1999-10-09,NaN,7f4ea9fb-f436-411e-ab34-e94750edfa93,8e2e1683-82da-48af-a740-80e9ce8e38ca,999969,Amlodipine 5 MG / Fentanyl 100 MCG / Olmesarta...,214.96,240,51590.40,59621000.0,Pain Relief
4,2003-07-31,NaN,3d8e57b2-3de5-4fb6-95b4-8370270c5cc2,82e9e0a1-3ef3-4cb8-b971-3e3de33652a1,904419,Alendronic acid 10 MG Oral Tablet,106.90,194,20738.60,NaN,NaN


In [23]:
medications_df['DESCRIPTION'].unique()

array(['Hydromorphone 325 MG',
       'diphenhydrAMINE Hydrochloride 25 MG Oral Tablet',
       'Ortho Tri-Cyclen 28 Day Pack',
       'Amlodipine 5 MG / Fentanyl 100 MCG / Olmesartan medoxomil 20 MG',
       'Alendronic acid 10 MG Oral Tablet', 'Camila 28 Day Pack',
       'Atenolol 50 MG / Chlorthalidone 25 MG Oral Tablet',
       'Chlorpheniramine Maleate 2 MG/ML Oral Solution',
       'Seasonique 91 Day Pack',
       '1 ML medroxyprogesterone acetate 150 MG/ML Injection',
       'Acetaminophen 325 MG Oral Tablet',
       'Acetaminophen 325 MG / HYDROcodone Bitartrate 7.5 MG Oral Tablet',
       'Ibuprofen 200 MG Oral Tablet', 'Jolivette 28 Day Pack',
       'Naproxen sodium 220 MG Oral Tablet',
       'Cefuroxime 250 MG Oral Tablet',
       'Acetaminophen 160 MG Oral Tablet',
       'Kyleena 19.5 MG Intrauterine System',
       'Ibuprofen 100 MG Oral Tablet', 'Mirena 52 MG Intrauterine System',
       'Natazia 28 Day Pack',
       'Abuse-Deterrent 12 HR Oxycodone Hydrochloride 15 M

In [24]:
#split medications to countable list, 
#create column with number of meds
medications_df['COUNT_CURRENT_MEDS'] = medications_df['DESCRIPTION'].str.replace(' / ', ', ').str.count(', ')+1

In [25]:
#drop unnecessary columns for final cohort
medications_df = medications_df.drop(['CODE', 'COST', 'TOTALCOST', 'REASONCODE', 'DISPENSES', 'REASONDESCRIPTION'], axis=1)

In [26]:
medications_df['START']= pd.to_datetime(medications_df['START'])
medications_df['STOP']= pd.to_datetime(medications_df['STOP'])

In [27]:
medications_df = medications_df.rename(columns = {'START': 'MED_START',
                                   'STOP': 'MED_STOP'})

In [28]:
medications_df.head()

,MED_START,MED_STOP,PATIENT,ENCOUNTER,DESCRIPTION,COUNT_CURRENT_MEDS
0,1997-09-20,NaT,2043c57c-1085-45a3-87e8-a223d5b6693d,3710b40a-bcc0-4c0f-a204-e47a5f3aec98,Hydromorphone 325 MG,1
1,1957-04-19,NaT,50f799aa-740c-4da9-ab85-49d5cbf4046b,64b4efb0-26ba-4c7f-91c1-64d8f62e49e3,diphenhydrAMINE Hydrochloride 25 MG Oral Tablet,1
2,2008-08-29,2009-08-24,553b00b2-347c-48ec-90ee-f59924ff50eb,ba6137e3-c8cd-42e6-9a02-eafa13d1cfa9,Ortho Tri-Cyclen 28 Day Pack,1
3,1999-10-09,NaT,7f4ea9fb-f436-411e-ab34-e94750edfa93,8e2e1683-82da-48af-a740-80e9ce8e38ca,Amlodipine 5 MG / Fentanyl 100 MCG / Olmesarta...,3
4,2003-07-31,NaT,3d8e57b2-3de5-4fb6-95b4-8370270c5cc2,82e9e0a1-3ef3-4cb8-b971-3e3de33652a1,Alendronic acid 10 MG Oral Tablet,1


#### patients

In [29]:
patients_df = pd.read_csv(r'C:\Users\katec\Documents\CHOP\analyst-take-home-task-master\analyst-take-home-task-master\datasets\patients.csv')

In [30]:
patients_df.head()

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,MAIDEN,MARITAL,RACE,ETHNICITY,GENDER,BIRTHPLACE,ADDRESS,CITY,STATE,ZIP
0,3d8e57b2-3de5-4fb6-95b4-8370270c5cc2,1943-03-11,NaN,999-86-7250,S99939389,X3970685X,Mrs.,Allyn942,Kreiger457,NaN,Bartoletti50,M,asian,asian_indian,F,Muhlenberg Pennsylvania US,372 Marks Heights Unit 20,Middle Paxton,Pennsylvania,NaN
1,7f4ea9fb-f436-411e-ab34-e94750edfa93,1980-09-28,NaN,999-90-4314,S99920355,X28211313X,Mr.,Kieth891,King743,NaN,NaN,M,white,irish,M,Overfield Pennsylvania US,428 Strosin Fort Suite 49,Limerick,Pennsylvania,NaN
2,553b00b2-347c-48ec-90ee-f59924ff50eb,1973-02-22,NaN,999-78-9189,S99980820,X9988931X,Mrs.,Domenica436,Fadel536,NaN,Labadie908,M,white,scottish,F,Wilkins Pennsylvania US,519 Ziemann Trail,Washington,Pennsylvania,15301.0
3,50f799aa-740c-4da9-ab85-49d5cbf4046b,1955-03-30,NaN,999-49-5162,S99961309,X17464073X,Mr.,Kelly223,Turner526,NaN,NaN,M,white,irish,M,Manor Pennsylvania US,158 Dickens Corner Suite 2,Manheim,Pennsylvania,17545.0
4,bcef3b7a-0380-4b7e-b2b9-2caf07e41f0e,2009-05-24,NaN,999-74-7736,NaN,NaN,NaN,Zack583,Purdy2,NaN,NaN,NaN,white,german,M,Cranberry Pennsylvania US,1000 Vandervort Run,Washington,Pennsylvania,15301.0


In [31]:
patients_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11864 entries, 0 to 11863
Data columns (total 20 columns):
Id            11864 non-null object
BIRTHDATE     11864 non-null object
DEATHDATE     1889 non-null object
SSN           11864 non-null object
DRIVERS       9883 non-null object
PASSPORT      9329 non-null object
PREFIX        9599 non-null object
FIRST         11864 non-null object
LAST          11864 non-null object
SUFFIX        100 non-null object
MAIDEN        3353 non-null object
MARITAL       8329 non-null object
RACE          11864 non-null object
ETHNICITY     11864 non-null object
GENDER        11864 non-null object
BIRTHPLACE    11864 non-null object
ADDRESS       11864 non-null object
CITY          11864 non-null object
STATE         11864 non-null object
ZIP           6946 non-null float64
dtypes: float64(1), object(19)
memory usage: 1.8+ MB


In [32]:
patients_df['BIRTHDATE']= pd.to_datetime(patients_df['BIRTHDATE'])
patients_df['DEATHDATE']= pd.to_datetime(patients_df['DEATHDATE'])

In [33]:
patients_df.dtypes['BIRTHDATE']

dtype('<M8[ns]')

In [34]:
#drop unnecessary columns for final cohort
patients_df = patients_df.drop(['SSN', 'DRIVERS', 'PASSPORT', 
                               'PREFIX', 'FIRST', 'LAST', 'SUFFIX',
                               'MAIDEN', 'MARITAL', 'RACE', 'ETHNICITY',
                               'GENDER', 'BIRTHPLACE', 'ADDRESS', 'CITY',
                               'STATE', 'ZIP'], axis=1)

#### procedures

In [35]:
procedures_df = pd.read_csv(r'C:\Users\katec\Documents\CHOP\analyst-take-home-task-master\analyst-take-home-task-master\datasets\procedures.csv')

In [36]:
procedures_df.head()

,DATE,PATIENT.x,ENCOUNTER,CODE.x,DESCRIPTION.x,COST.x,REASONCODE.x,REASONDESCRIPTION.x
0,2009-09-17,553b00b2-347c-48ec-90ee-f59924ff50eb,d08babad-74ed-4490-8830-24af7310cac7,76601001,Intramuscular injection,1858.49,NaN,NaN
1,2009-12-17,553b00b2-347c-48ec-90ee-f59924ff50eb,fbfd3904-6135-4572-a181-cb4b64878ee4,76601001,Intramuscular injection,2972.12,NaN,NaN
2,2010-03-18,553b00b2-347c-48ec-90ee-f59924ff50eb,07d30215-3df2-4a12-8e04-dfd6c04024df,76601001,Intramuscular injection,2406.65,NaN,NaN
3,2010-06-17,553b00b2-347c-48ec-90ee-f59924ff50eb,23aee72d-fae1-48ad-92a3-375f636f7dbb,76601001,Intramuscular injection,2410.85,NaN,NaN
4,2010-03-29,50f799aa-740c-4da9-ab85-49d5cbf4046b,c89c11eb-f144-421f-8b3e-a1874f11203e,73761001,Colonoscopy,8577.04,NaN,NaN


In [37]:
procedures_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324889 entries, 0 to 324888
Data columns (total 8 columns):
DATE                   324889 non-null object
PATIENT.x              324889 non-null object
ENCOUNTER              324889 non-null object
CODE.x                 324889 non-null object
DESCRIPTION.x          324889 non-null object
COST.x                 324889 non-null float64
REASONCODE.x           185489 non-null float64
REASONDESCRIPTION.x    186699 non-null object
dtypes: float64(2), object(6)
memory usage: 19.8+ MB


In [38]:
procedures_df['DESCRIPTION.x'].unique()

array(['Intramuscular injection', 'Colonoscopy',
       'Documentation of current medications', 'Standard pregnancy test',
       'Ultrasound scan for fetal viability',
       'Evaluation of uterine fundal height',
       'Auscultation of the fetal heart', 'Blood typing  RH typing',
       'Hemoglobin / Hematocrit / Platelet count',
       'Hepatitis B Surface Antigen Measurement',
       'Human immunodeficiency virus antigen test',
       'Chlamydia antigen test', 'Gonorrhea infection test',
       'Syphilis infection test', 'Urine culture',
       'Cytopathology procedure  preparation of smear  genital source',
       'Urine screening test for diabetes', 'Hepatitis C antibody test',
       'Rubella screening',
       'Measurement of Varicella-zoster virus antibody',
       'Skin test for tuberculosis', 'Urine protein test',
       'Physical examination of mother',
       'Screening for chromosomal aneuploidy in prenatal amniotic fluid',
       'Admission to trauma surgery department'

In [39]:
procedures_df[procedures_df['DESCRIPTION.x'].str.contains('Drug overdose', na=False)]

,DATE,PATIENT.x,ENCOUNTER,CODE.x,DESCRIPTION.x,COST.x,REASONCODE.x,REASONDESCRIPTION.x


##### comment/assumptions:
no 'drug overdose' found in this column

In [40]:
procedures_df['REASONDESCRIPTION.x'].unique()

array([nan, 'Normal pregnancy', 'Bullet wound',
       'Malignant neoplasm of breast (disorder)', 'Fracture of forearm',
       'Acute bronchitis (disorder)',
       'Acute viral pharyngitis (disorder)', 'Laceration of forearm',
       'Laceration of foot', 'Fracture of clavicle', 'Fracture of rib',
       'Polyp of colon', 'Recurrent rectal polyp',
       'Malignant tumor of colon', 'Laceration of hand',
       'Overlapping malignant neoplasm of colon',
       'Chronic obstructive bronchitis (disorder)',
       'Closed fracture of hip', 'Laceration of thigh',
       'Childhood asthma', 'Streptococcal sore throat (disorder)',
       'Facial laceration', 'Fracture subluxation of wrist',
       'Rupture of patellar tendon', 'Neoplasm of prostate',
       'Atrial Fibrillation', 'Cardiac Arrest',
       'Suspected lung cancer (situation)',
       'Non-small cell carcinoma of lung  TNM stage 1 (disorder)',
       'Fracture of ankle', 'Pulmonary emphysema (disorder)', 'Stroke',
       'Child

In [41]:
procedures_df[procedures_df['REASONDESCRIPTION.x'].str.contains('Drug overdose', na=False)]

,DATE,PATIENT.x,ENCOUNTER,CODE.x,DESCRIPTION.x,COST.x,REASONCODE.x,REASONDESCRIPTION.x


##### comment/assumptions:
no 'drug overdose' found in this column

In [42]:
procedures_df = procedures_df.drop(['DESCRIPTION.x', 'REASONDESCRIPTION.x', 'CODE.x', 'COST.x', 'REASONCODE.x'], axis=1)

### merge to create cohort

In [43]:
encounters_df = encounters_df.rename(columns={'Id':'ENCOUNTER_ID', 'PATIENT': 'PATIENT_ID', 'START': 'HOSPITAL_ENCOUNTER_DATE'})
procedures_df = procedures_df.rename(columns={'ENCOUNTER':'ENCOUNTER_ID', 'PATIENT.x': 'PATIENT_ID'})

In [44]:
merged_df = pd.merge(encounters_df, procedures_df, on=['ENCOUNTER_ID', 'PATIENT_ID'], how='left')

In [45]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 549871 entries, 0 to 549870
Data columns (total 7 columns):
ENCOUNTER_ID               549871 non-null object
HOSPITAL_ENCOUNTER_DATE    549871 non-null datetime64[ns]
STOP                       549871 non-null datetime64[ns]
PATIENT_ID                 549871 non-null object
ENCOUNTERCLASS             549871 non-null object
REASONDESCRIPTION          249253 non-null object
DATE                       324889 non-null object
dtypes: datetime64[ns](2), object(5)
memory usage: 33.6+ MB


In [46]:
patients_df = patients_df.rename(columns={'Id':'PATIENT_ID'})

In [47]:
merged_df = pd.merge(merged_df, patients_df, on=['PATIENT_ID'], how='left')

In [48]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 549871 entries, 0 to 549870
Data columns (total 9 columns):
ENCOUNTER_ID               549871 non-null object
HOSPITAL_ENCOUNTER_DATE    549871 non-null datetime64[ns]
STOP                       549871 non-null datetime64[ns]
PATIENT_ID                 549871 non-null object
ENCOUNTERCLASS             549871 non-null object
REASONDESCRIPTION          249253 non-null object
DATE                       324889 non-null object
BIRTHDATE                  549871 non-null datetime64[ns]
DEATHDATE                  118800 non-null datetime64[ns]
dtypes: datetime64[ns](4), object(5)
memory usage: 42.0+ MB


In [49]:
medications_df = medications_df.rename(columns={'ENCOUNTER':'ENCOUNTER_ID', 'PATIENT':'PATIENT_ID', 'DESCRIPTION': 'MED_DESCRIPTION'})

In [50]:
merged_df = pd.merge(merged_df, medications_df, on=['ENCOUNTER_ID', 'PATIENT_ID'], how='left')

In [51]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 574006 entries, 0 to 574005
Data columns (total 13 columns):
ENCOUNTER_ID               574006 non-null object
HOSPITAL_ENCOUNTER_DATE    574006 non-null datetime64[ns]
STOP                       574006 non-null datetime64[ns]
PATIENT_ID                 574006 non-null object
ENCOUNTERCLASS             574006 non-null object
REASONDESCRIPTION          257011 non-null object
DATE                       344038 non-null object
BIRTHDATE                  574006 non-null datetime64[ns]
DEATHDATE                  133623 non-null datetime64[ns]
MED_START                  132393 non-null datetime64[ns]
MED_STOP                   109796 non-null datetime64[ns]
MED_DESCRIPTION            132393 non-null object
COUNT_CURRENT_MEDS         132393 non-null float64
dtypes: datetime64[ns](6), float64(1), object(6)
memory usage: 61.3+ MB


In [52]:
allergies_df = allergies_df.rename(columns={'ENCOUNTER':'ENCOUNTER_ID', 'PATIENT':'PATIENT_ID'})

In [53]:
merged_df = pd.merge(merged_df, allergies_df, on=['ENCOUNTER_ID', 'PATIENT_ID'], how='left')

In [54]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 580663 entries, 0 to 580662
Data columns (total 16 columns):
ENCOUNTER_ID               580663 non-null object
HOSPITAL_ENCOUNTER_DATE    580663 non-null datetime64[ns]
STOP_x                     580663 non-null datetime64[ns]
PATIENT_ID                 580663 non-null object
ENCOUNTERCLASS             580663 non-null object
REASONDESCRIPTION          257011 non-null object
DATE                       346128 non-null object
BIRTHDATE                  580663 non-null datetime64[ns]
DEATHDATE                  133862 non-null datetime64[ns]
MED_START                  138296 non-null datetime64[ns]
MED_STOP                   109796 non-null datetime64[ns]
MED_DESCRIPTION            138296 non-null object
COUNT_CURRENT_MEDS         138296 non-null float64
START                      8661 non-null object
STOP_y                     964 non-null object
DESCRIPTION                8661 non-null object
dtypes: datetime64[ns](6), float64(1), object(9

In [55]:
merged_df = merged_df.drop_duplicates()

In [56]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 436167 entries, 0 to 580662
Data columns (total 16 columns):
ENCOUNTER_ID               436167 non-null object
HOSPITAL_ENCOUNTER_DATE    436167 non-null datetime64[ns]
STOP_x                     436167 non-null datetime64[ns]
PATIENT_ID                 436167 non-null object
ENCOUNTERCLASS             436167 non-null object
REASONDESCRIPTION          140509 non-null object
DATE                       201632 non-null object
BIRTHDATE                  436167 non-null datetime64[ns]
DEATHDATE                  108385 non-null datetime64[ns]
MED_START                  117837 non-null datetime64[ns]
MED_STOP                   91671 non-null datetime64[ns]
MED_DESCRIPTION            117837 non-null object
COUNT_CURRENT_MEDS         117837 non-null float64
START                      8661 non-null object
STOP_y                     964 non-null object
DESCRIPTION                8661 non-null object
dtypes: datetime64[ns](6), float64(1), object(9)

In [57]:
merged_df.head(50)

,ENCOUNTER_ID,HOSPITAL_ENCOUNTER_DATE,STOP_x,PATIENT_ID,ENCOUNTERCLASS,REASONDESCRIPTION,DATE,BIRTHDATE,DEATHDATE,MED_START,MED_STOP,MED_DESCRIPTION,COUNT_CURRENT_MEDS,START,STOP_y,DESCRIPTION
0,2590963f-222b-4870-b8c7-86d07f4d44b3,1959-06-18 09:43:49,1959-06-25 21:37:31,3d8e57b2-3de5-4fb6-95b4-8370270c5cc2,ambulatory,Normal pregnancy,NaN,1943-03-11,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
1,ba6137e3-c8cd-42e6-9a02-eafa13d1cfa9,2008-08-29 19:14:19,2008-09-07 13:33:07,553b00b2-347c-48ec-90ee-f59924ff50eb,outpatient,NaN,NaN,1973-02-22,NaT,2008-08-29,2009-08-24,Ortho Tri-Cyclen 28 Day Pack,1.0,NaN,NaN,NaN
2,10aa7678-5c84-4ce9-81af-09272307c1f0,1999-07-12 03:05:41,1999-07-20 11:18:24,7f4ea9fb-f436-411e-ab34-e94750edfa93,wellness,NaN,NaN,1980-09-28,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
3,3710b40a-bcc0-4c0f-a204-e47a5f3aec98,1997-09-20 16:09:36,1997-09-30 22:43:32,2043c57c-1085-45a3-87e8-a223d5b6693d,wellness,NaN,NaN,1978-09-30,NaT,1997-09-20,NaT,Hydromorphone 325 MG,1.0,NaN,NaN,NaN
4,92078752-a89c-4931-b95e-0f8df485e9f6,1963-09-26 09:43:49,1963-10-05 03:20:42,3d8e57b2-3de5-4fb6-95b4-8370270c5cc2,ambulatory,Normal pregnancy,NaN,1943-03-11,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
5,769426dd-84e1-4a59-8588-5c656dc44c3f,2009-08-30 20:39:02,2009-09-08 23:56:41,bcef3b7a-0380-4b7e-b2b9-2caf07e41f0e,wellness,NaN,NaN,2009-05-24,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
6,64b4efb0-26ba-4c7f-91c1-64d8f62e49e3,1957-04-05 07:05:09,1957-04-12 15:09:36,50f799aa-740c-4da9-ab85-49d5cbf4046b,ambulatory,Seasonal allergic rhinitis,NaN,1955-03-30,NaT,1957-04-19,NaT,diphenhydrAMINE Hydrochloride 25 MG Oral Tablet,1.0,NaN,NaN,NaN
7,a0f0942e-f7ed-48ad-8082-25f32834e3d9,1957-04-26 07:05:09,1957-05-06 05:23:01,50f799aa-740c-4da9-ab85-49d5cbf4046b,ambulatory,NaN,NaN,1955-03-30,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
8,d08babad-74ed-4490-8830-24af7310cac7,2009-09-17 19:14:19,2009-09-24 15:37:34,553b00b2-347c-48ec-90ee-f59924ff50eb,outpatient,NaN,2009-09-17,1973-02-22,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
9,e5092bf0-8ea9-4bb4-87d4-c1c7826cd52b,2010-02-19 12:30:21,2010-02-27 07:13:56,12645cb2-8205-4627-b7d0-4eca7a9a90ae,wellness,NaN,NaN,1997-01-17,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN


In [58]:
merged_df.isnull().sum()

ENCOUNTER_ID                    0
HOSPITAL_ENCOUNTER_DATE         0
STOP_x                          0
PATIENT_ID                      0
ENCOUNTERCLASS                  0
REASONDESCRIPTION          295658
DATE                       234535
BIRTHDATE                       0
DEATHDATE                  327782
MED_START                  318330
MED_STOP                   344496
MED_DESCRIPTION            318330
COUNT_CURRENT_MEDS         318330
START                      427506
STOP_y                     435203
DESCRIPTION                427506
dtype: int64

##### comment/assumptions
will deal with null values once cohort is selected

### identify:

1. The patient’s visit is an encounter for drug overdose

##### assumption:
patient's visit is an 'inpatient' OR 'emergency' visit

In [59]:
merged_df = merged_df[(merged_df['ENCOUNTERCLASS'] == 'inpatient')|(merged_df['ENCOUNTERCLASS'] =='emergency')]

In [60]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49551 entries, 17 to 580629
Data columns (total 16 columns):
ENCOUNTER_ID               49551 non-null object
HOSPITAL_ENCOUNTER_DATE    49551 non-null datetime64[ns]
STOP_x                     49551 non-null datetime64[ns]
PATIENT_ID                 49551 non-null object
ENCOUNTERCLASS             49551 non-null object
REASONDESCRIPTION          19501 non-null object
DATE                       31010 non-null object
BIRTHDATE                  49551 non-null datetime64[ns]
DEATHDATE                  22566 non-null datetime64[ns]
MED_START                  26964 non-null datetime64[ns]
MED_STOP                   25968 non-null datetime64[ns]
MED_DESCRIPTION            26964 non-null object
COUNT_CURRENT_MEDS         26964 non-null float64
START                      0 non-null object
STOP_y                     0 non-null object
DESCRIPTION                0 non-null object
dtypes: datetime64[ns](6), float64(1), object(9)
memory usage: 6.4+ 

In [61]:
cohort_df = merged_df[merged_df['REASONDESCRIPTION'].str.contains('Drug overdose', na=False)]

In [62]:
cohort_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2449 entries, 403 to 579688
Data columns (total 16 columns):
ENCOUNTER_ID               2449 non-null object
HOSPITAL_ENCOUNTER_DATE    2449 non-null datetime64[ns]
STOP_x                     2449 non-null datetime64[ns]
PATIENT_ID                 2449 non-null object
ENCOUNTERCLASS             2449 non-null object
REASONDESCRIPTION          2449 non-null object
DATE                       15 non-null object
BIRTHDATE                  2449 non-null datetime64[ns]
DEATHDATE                  452 non-null datetime64[ns]
MED_START                  3 non-null datetime64[ns]
MED_STOP                   3 non-null datetime64[ns]
MED_DESCRIPTION            3 non-null object
COUNT_CURRENT_MEDS         3 non-null float64
START                      0 non-null object
STOP_y                     0 non-null object
DESCRIPTION                0 non-null object
dtypes: datetime64[ns](6), float64(1), object(9)
memory usage: 325.3+ KB


In [63]:
cohort_df.head(50)

,ENCOUNTER_ID,HOSPITAL_ENCOUNTER_DATE,STOP_x,PATIENT_ID,ENCOUNTERCLASS,REASONDESCRIPTION,DATE,BIRTHDATE,DEATHDATE,MED_START,MED_STOP,MED_DESCRIPTION,COUNT_CURRENT_MEDS,START,STOP_y,DESCRIPTION
403,2a917920-2701-49f0-9340-827320eef76d,2003-03-31 21:50:51,2003-04-08 13:20:43,708b81c9-21a9-411a-aae4-052ef8925b97,emergency,Drug overdose,NaN,1986-10-28,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
421,22874b3d-0873-40e4-a455-897d51d593f7,2012-02-18 21:50:51,2012-02-28 21:12:17,708b81c9-21a9-411a-aae4-052ef8925b97,emergency,Drug overdose,NaN,1986-10-28,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
425,134c5ee3-1b72-4e3e-a44a-3efebe77603b,2013-08-03 21:50:51,2013-08-13 07:44:52,708b81c9-21a9-411a-aae4-052ef8925b97,emergency,Drug overdose,NaN,1986-10-28,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
429,6125f147-72d4-48a4-9813-2a9dc878f055,2014-12-08 21:50:51,2014-12-17 12:25:27,708b81c9-21a9-411a-aae4-052ef8925b97,emergency,Drug overdose,NaN,1986-10-28,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
438,f837dcf8-af7d-43b8-b5c2-773d494617ab,2015-08-31 21:50:51,2015-09-08 12:04:08,708b81c9-21a9-411a-aae4-052ef8925b97,emergency,Drug overdose,NaN,1986-10-28,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
1803,2b235f52-2da8-4d48-b100-dfe72e698771,1997-07-06 11:26:13,1997-07-15 12:25:23,722270fe-f8dd-46e6-aace-756450258911,emergency,Drug overdose,NaN,1955-04-05,2016-04-21,NaT,NaT,NaN,NaN,NaN,NaN,NaN
1816,068d0cbf-1625-48ae-9b4c-c32c208489f1,2007-03-17 11:26:13,2007-03-22 20:51:44,722270fe-f8dd-46e6-aace-756450258911,emergency,Drug overdose,2007-03-20,1955-04-05,2016-04-21,NaT,NaT,NaN,NaN,NaN,NaN,NaN
1817,cfa60809-f10a-47d0-83fd-87f964989490,2007-07-07 11:26:13,2007-07-14 10:18:31,722270fe-f8dd-46e6-aace-756450258911,emergency,Drug overdose,NaN,1955-04-05,2016-04-21,NaT,NaT,NaN,NaN,NaN,NaN,NaN
1855,7b843d12-bfe0-44a8-aa83-3cd98d7344b5,2012-06-07 11:26:13,2012-06-18 01:04:13,722270fe-f8dd-46e6-aace-756450258911,emergency,Drug overdose,NaN,1955-04-05,2016-04-21,NaT,NaT,NaN,NaN,NaN,NaN,NaN
1857,94d455e3-6cdc-4bc6-9555-5e05f817d4da,2012-09-20 11:26:13,2012-09-30 22:39:40,722270fe-f8dd-46e6-aace-756450258911,emergency,Drug overdose,NaN,1955-04-05,2016-04-21,NaT,NaT,NaN,NaN,NaN,NaN,NaN


In [64]:
cohort_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2449 entries, 403 to 579688
Data columns (total 16 columns):
ENCOUNTER_ID               2449 non-null object
HOSPITAL_ENCOUNTER_DATE    2449 non-null datetime64[ns]
STOP_x                     2449 non-null datetime64[ns]
PATIENT_ID                 2449 non-null object
ENCOUNTERCLASS             2449 non-null object
REASONDESCRIPTION          2449 non-null object
DATE                       15 non-null object
BIRTHDATE                  2449 non-null datetime64[ns]
DEATHDATE                  452 non-null datetime64[ns]
MED_START                  3 non-null datetime64[ns]
MED_STOP                   3 non-null datetime64[ns]
MED_DESCRIPTION            3 non-null object
COUNT_CURRENT_MEDS         3 non-null float64
START                      0 non-null object
STOP_y                     0 non-null object
DESCRIPTION                0 non-null object
dtypes: datetime64[ns](6), float64(1), object(9)
memory usage: 325.3+ KB


### identify
2. The hospital encounter occurs after July 15, 1999
3. The patient’s age at time of encounter is between 18 and 35 (Patient is considered to be 35 until turning 36)

In [65]:
#hospital encounter occurs after July 15, 1999
cohort_df = cohort_df[(cohort_df['HOSPITAL_ENCOUNTER_DATE'] > '1999-07-15 12:59:59')]

In [66]:
#calculate patient's age
cohort_df['AGE_AT_VISIT'] = cohort_df['HOSPITAL_ENCOUNTER_DATE'].dt.date - cohort_df['BIRTHDATE'].dt.date
cohort_df['AGE_AT_VISIT'] = cohort_df['AGE_AT_VISIT']/np.timedelta64(1,'Y')

In [67]:
#select ages between 18 - 35
cohort_df = cohort_df[(cohort_df['AGE_AT_VISIT'] >= 18)& (cohort_df['AGE_AT_VISIT'] < 36)]

In [68]:
cohort_df.head(50)

,ENCOUNTER_ID,HOSPITAL_ENCOUNTER_DATE,STOP_x,PATIENT_ID,ENCOUNTERCLASS,REASONDESCRIPTION,DATE,BIRTHDATE,DEATHDATE,MED_START,MED_STOP,MED_DESCRIPTION,COUNT_CURRENT_MEDS,START,STOP_y,DESCRIPTION,AGE_AT_VISIT
421,22874b3d-0873-40e4-a455-897d51d593f7,2012-02-18 21:50:51,2012-02-28 21:12:17,708b81c9-21a9-411a-aae4-052ef8925b97,emergency,Drug overdose,NaN,1986-10-28,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,25.309212
425,134c5ee3-1b72-4e3e-a44a-3efebe77603b,2013-08-03 21:50:51,2013-08-13 07:44:52,708b81c9-21a9-411a-aae4-052ef8925b97,emergency,Drug overdose,NaN,1986-10-28,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,26.765779
429,6125f147-72d4-48a4-9813-2a9dc878f055,2014-12-08 21:50:51,2014-12-17 12:25:27,708b81c9-21a9-411a-aae4-052ef8925b97,emergency,Drug overdose,NaN,1986-10-28,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,28.112829
438,f837dcf8-af7d-43b8-b5c2-773d494617ab,2015-08-31 21:50:51,2015-09-08 12:04:08,708b81c9-21a9-411a-aae4-052ef8925b97,emergency,Drug overdose,NaN,1986-10-28,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,28.841112
4523,010594a6-a6ff-4875-9a9e-6b213367cde0,2018-12-10 00:59:35,2018-12-20 22:57:39,65b093e4-b353-447a-b9ae-290567f1c950,emergency,Drug overdose,NaN,1995-12-03,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,23.020322
7301,06b99d50-cb73-4f85-89be-49c5d0d39eba,2009-07-23 01:30:34,2009-07-31 14:50:01,dacea80b-75dd-42d6-a5c0-be369c3e4ebf,emergency,Drug overdose,NaN,1979-12-11,2009-08-24,NaT,NaT,NaN,NaN,NaN,NaN,NaN,29.615940
7302,1deb51e9-ef0b-4013-b46f-f8efcd836842,2009-08-20 01:30:34,2009-08-24 21:37:59,dacea80b-75dd-42d6-a5c0-be369c3e4ebf,emergency,Drug overdose,NaN,1979-12-11,2009-08-24,NaT,NaT,NaN,NaN,NaN,NaN,NaN,29.692601
10650,f8e08525-2b6a-451e-a766-112a662549b1,2003-12-13 09:39:47,2003-12-22 20:26:44,ad9e95d4-f88b-4544-b5c7-12087fe1fab0,emergency,Drug overdose,NaN,1976-11-04,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,27.105279
10940,ca5938ad-3be6-4323-a92e-59be95caadae,2017-01-11 08:48:34,2017-01-20 05:57:10,9b5505e9-0806-47ad-8a51-adc39c20b468,emergency,Drug overdose,NaN,1998-11-26,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,18.127682
12885,649a8b1d-0bce-43ea-8e7d-6e7555fb8dee,2014-12-05 07:57:48,2014-12-14 19:31:05,bad7331f-c92c-4085-9313-8a2c59bd66fa,emergency,Drug overdose,NaN,1986-01-13,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,28.893133


## Part 2: Create additional fields

With your drug overdose encounter, create the following indicators:

    DEATH_AT_VISIT_IND: 1 if patient died during the drug overdose encounter, 0 if the patient died at a different time
    COUNT_CURRENT_MEDS: Count of active medications at the start of the drug overdose encounter
    CURRENT_OPIOID_IND: 1 if the patient had at least one active medication at the start of the overdose encounter that is on the Opioids List (provided below), 0 if not
    READMISSION_90_DAY_IND: 1 if the visit resulted in a subsequent drug overdose readmission within 90 days, 0 if not
    READMISSION_30_DAY_IND: 1 if the visit resulted in a subsequent drug overdose readmission within 30 days, 0 if not overdose encounter, 0 if not
    FIRST_READMISSION_DATE: The date of the index visit's first readmission for drug overdose. Field should be left as N/A if no readmission for drug overdose within 90 days

#### DEATH_AT_VISIT_IND: 
1 if patient died during the drug overdose encounter, 
0 if the patient died at a different time

In [69]:
cohort_df.loc[cohort_df['DEATHDATE'] == 'NaT', 'DEATH_AT_VISIT_IND'] = 'NaT'

In [70]:
cohort_df['DEATH_AT_VISIT_IND'] = np.where(cohort_df['DEATHDATE'].dt.date== cohort_df['STOP_x'].dt.date, 1,
                           np.where(cohort_df['DEATHDATE']!= 'NaT',0, 'NaT'))

In [71]:
cohort_df.head(50)

,ENCOUNTER_ID,HOSPITAL_ENCOUNTER_DATE,STOP_x,PATIENT_ID,ENCOUNTERCLASS,REASONDESCRIPTION,DATE,BIRTHDATE,DEATHDATE,MED_START,MED_STOP,MED_DESCRIPTION,COUNT_CURRENT_MEDS,START,STOP_y,DESCRIPTION,AGE_AT_VISIT,DEATH_AT_VISIT_IND
421,22874b3d-0873-40e4-a455-897d51d593f7,2012-02-18 21:50:51,2012-02-28 21:12:17,708b81c9-21a9-411a-aae4-052ef8925b97,emergency,Drug overdose,NaN,1986-10-28,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,25.309212,0
425,134c5ee3-1b72-4e3e-a44a-3efebe77603b,2013-08-03 21:50:51,2013-08-13 07:44:52,708b81c9-21a9-411a-aae4-052ef8925b97,emergency,Drug overdose,NaN,1986-10-28,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,26.765779,0
429,6125f147-72d4-48a4-9813-2a9dc878f055,2014-12-08 21:50:51,2014-12-17 12:25:27,708b81c9-21a9-411a-aae4-052ef8925b97,emergency,Drug overdose,NaN,1986-10-28,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,28.112829,0
438,f837dcf8-af7d-43b8-b5c2-773d494617ab,2015-08-31 21:50:51,2015-09-08 12:04:08,708b81c9-21a9-411a-aae4-052ef8925b97,emergency,Drug overdose,NaN,1986-10-28,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,28.841112,0
4523,010594a6-a6ff-4875-9a9e-6b213367cde0,2018-12-10 00:59:35,2018-12-20 22:57:39,65b093e4-b353-447a-b9ae-290567f1c950,emergency,Drug overdose,NaN,1995-12-03,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,23.020322,0
7301,06b99d50-cb73-4f85-89be-49c5d0d39eba,2009-07-23 01:30:34,2009-07-31 14:50:01,dacea80b-75dd-42d6-a5c0-be369c3e4ebf,emergency,Drug overdose,NaN,1979-12-11,2009-08-24,NaT,NaT,NaN,NaN,NaN,NaN,NaN,29.615940,0
7302,1deb51e9-ef0b-4013-b46f-f8efcd836842,2009-08-20 01:30:34,2009-08-24 21:37:59,dacea80b-75dd-42d6-a5c0-be369c3e4ebf,emergency,Drug overdose,NaN,1979-12-11,2009-08-24,NaT,NaT,NaN,NaN,NaN,NaN,NaN,29.692601,1
10650,f8e08525-2b6a-451e-a766-112a662549b1,2003-12-13 09:39:47,2003-12-22 20:26:44,ad9e95d4-f88b-4544-b5c7-12087fe1fab0,emergency,Drug overdose,NaN,1976-11-04,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,27.105279,0
10940,ca5938ad-3be6-4323-a92e-59be95caadae,2017-01-11 08:48:34,2017-01-20 05:57:10,9b5505e9-0806-47ad-8a51-adc39c20b468,emergency,Drug overdose,NaN,1998-11-26,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,18.127682,0
12885,649a8b1d-0bce-43ea-8e7d-6e7555fb8dee,2014-12-05 07:57:48,2014-12-14 19:31:05,bad7331f-c92c-4085-9313-8a2c59bd66fa,emergency,Drug overdose,NaN,1986-01-13,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,28.893133,0


#### COUNT_CURRENT_MEDS: 
Count of active medications at the start of the drug overdose encounter
##### comment/assumption:
this item was created prior to merge of files

In [72]:
# fill null with 0 active meds
cohort_df.loc[:, 'COUNT_CURRENT_MEDS'] = cohort_df.loc[:, 'COUNT_CURRENT_MEDS'].fillna(0)

In [73]:
cohort_df.head()

,ENCOUNTER_ID,HOSPITAL_ENCOUNTER_DATE,STOP_x,PATIENT_ID,ENCOUNTERCLASS,REASONDESCRIPTION,DATE,BIRTHDATE,DEATHDATE,MED_START,MED_STOP,MED_DESCRIPTION,COUNT_CURRENT_MEDS,START,STOP_y,DESCRIPTION,AGE_AT_VISIT,DEATH_AT_VISIT_IND
421,22874b3d-0873-40e4-a455-897d51d593f7,2012-02-18 21:50:51,2012-02-28 21:12:17,708b81c9-21a9-411a-aae4-052ef8925b97,emergency,Drug overdose,NaN,1986-10-28,NaT,NaT,NaT,NaN,0.0,NaN,NaN,NaN,25.309212,0
425,134c5ee3-1b72-4e3e-a44a-3efebe77603b,2013-08-03 21:50:51,2013-08-13 07:44:52,708b81c9-21a9-411a-aae4-052ef8925b97,emergency,Drug overdose,NaN,1986-10-28,NaT,NaT,NaT,NaN,0.0,NaN,NaN,NaN,26.765779,0
429,6125f147-72d4-48a4-9813-2a9dc878f055,2014-12-08 21:50:51,2014-12-17 12:25:27,708b81c9-21a9-411a-aae4-052ef8925b97,emergency,Drug overdose,NaN,1986-10-28,NaT,NaT,NaT,NaN,0.0,NaN,NaN,NaN,28.112829,0
438,f837dcf8-af7d-43b8-b5c2-773d494617ab,2015-08-31 21:50:51,2015-09-08 12:04:08,708b81c9-21a9-411a-aae4-052ef8925b97,emergency,Drug overdose,NaN,1986-10-28,NaT,NaT,NaT,NaN,0.0,NaN,NaN,NaN,28.841112,0
4523,010594a6-a6ff-4875-9a9e-6b213367cde0,2018-12-10 00:59:35,2018-12-20 22:57:39,65b093e4-b353-447a-b9ae-290567f1c950,emergency,Drug overdose,NaN,1995-12-03,NaT,NaT,NaT,NaN,0.0,NaN,NaN,NaN,23.020322,0


#### CURRENT_OPIOID_IND: 
1 if the patient had at least one active medication at the start of the overdose encounter that is on the Opioids List (provided below): 

    Hydromorphone 325Mg
    Fentanyl – 100 MCG
    Oxycodone-acetaminophen 100 Ml

0 if not

In [74]:
cohort_df['CURRENT_OPIOID_IND'] = np.where(cohort_df['MED_DESCRIPTION'].str.contains('Hydromorphone 325Mg')| cohort_df['MED_DESCRIPTION'].str.contains('Fentanyl – 100 MCG')| cohort_df['MED_DESCRIPTION'].str.contains('Oxycodone-acetaminophen 100 Ml'), 1, 0)

In [75]:
cohort_df.head()

,ENCOUNTER_ID,HOSPITAL_ENCOUNTER_DATE,STOP_x,PATIENT_ID,ENCOUNTERCLASS,REASONDESCRIPTION,DATE,BIRTHDATE,DEATHDATE,MED_START,MED_STOP,MED_DESCRIPTION,COUNT_CURRENT_MEDS,START,STOP_y,DESCRIPTION,AGE_AT_VISIT,DEATH_AT_VISIT_IND,CURRENT_OPIOID_IND
421,22874b3d-0873-40e4-a455-897d51d593f7,2012-02-18 21:50:51,2012-02-28 21:12:17,708b81c9-21a9-411a-aae4-052ef8925b97,emergency,Drug overdose,NaN,1986-10-28,NaT,NaT,NaT,NaN,0.0,NaN,NaN,NaN,25.309212,0,0
425,134c5ee3-1b72-4e3e-a44a-3efebe77603b,2013-08-03 21:50:51,2013-08-13 07:44:52,708b81c9-21a9-411a-aae4-052ef8925b97,emergency,Drug overdose,NaN,1986-10-28,NaT,NaT,NaT,NaN,0.0,NaN,NaN,NaN,26.765779,0,0
429,6125f147-72d4-48a4-9813-2a9dc878f055,2014-12-08 21:50:51,2014-12-17 12:25:27,708b81c9-21a9-411a-aae4-052ef8925b97,emergency,Drug overdose,NaN,1986-10-28,NaT,NaT,NaT,NaN,0.0,NaN,NaN,NaN,28.112829,0,0
438,f837dcf8-af7d-43b8-b5c2-773d494617ab,2015-08-31 21:50:51,2015-09-08 12:04:08,708b81c9-21a9-411a-aae4-052ef8925b97,emergency,Drug overdose,NaN,1986-10-28,NaT,NaT,NaT,NaN,0.0,NaN,NaN,NaN,28.841112,0,0
4523,010594a6-a6ff-4875-9a9e-6b213367cde0,2018-12-10 00:59:35,2018-12-20 22:57:39,65b093e4-b353-447a-b9ae-290567f1c950,emergency,Drug overdose,NaN,1995-12-03,NaT,NaT,NaT,NaN,0.0,NaN,NaN,NaN,23.020322,0,0


#### READMISSION_90_DAY_IND: 
1 if the visit resulted in a subsequent drug overdose readmission within 90 days, 
0 if not   

In [76]:
cohort_df['READMISSION_90_DAY_IND'] = np.nan

#### READMISSION_30_DAY_IND: 
1 if the visit resulted in a subsequent drug overdose readmission within 30 days, 
0 if not overdose encounter, 
0 if not

In [77]:
cohort_df['READMISSION_30_DAY_IND'] = np.nan

#### FIRST_READMISSION_DATE: 
The date of the index visit's first readmission for drug overdose. Field should be left as N/A if no readmission for drug overdose within 90 days

In [78]:
cohort_df['FIRST_READMISSION_DATE'] = np.nan

## Part 3: Export the data to a CSV file

Export a dataset containing these required fields:

In [79]:
cohort_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 431 entries, 421 to 574204
Data columns (total 22 columns):
ENCOUNTER_ID               431 non-null object
HOSPITAL_ENCOUNTER_DATE    431 non-null datetime64[ns]
STOP_x                     431 non-null datetime64[ns]
PATIENT_ID                 431 non-null object
ENCOUNTERCLASS             431 non-null object
REASONDESCRIPTION          431 non-null object
DATE                       1 non-null object
BIRTHDATE                  431 non-null datetime64[ns]
DEATHDATE                  71 non-null datetime64[ns]
MED_START                  0 non-null datetime64[ns]
MED_STOP                   0 non-null datetime64[ns]
MED_DESCRIPTION            0 non-null object
COUNT_CURRENT_MEDS         431 non-null float64
START                      0 non-null object
STOP_y                     0 non-null object
DESCRIPTION                0 non-null object
AGE_AT_VISIT               431 non-null float64
DEATH_AT_VISIT_IND         431 non-null object
CURRENT_O

In [80]:
cohort_final = cohort_df.drop(['STOP_x', 'ENCOUNTERCLASS', 'REASONDESCRIPTION', 
                               'DATE', 'BIRTHDATE', 'DEATHDATE', 'MED_START',
                               'MED_STOP', 'MED_DESCRIPTION', 'START', 'STOP_y',
                               'DESCRIPTION'], axis=1)

In [81]:
column_names = ['PATIENT_ID', 'ENCOUNTER_ID', 'HOSPITAL_ENCOUNTER_DATE',
                'AGE_AT_VISIT', 'DEATH_AT_VISIT_IND', 'COUNT_CURRENT_MEDS',
                'CURRENT_OPIOID_IND', 'READMISSION_90_DAY_IND', 
                'READMISSION_30_DAY_IND', 'FIRST_READMISSION_DATE']


cohort_final = cohort_final.reindex(columns=column_names)

In [82]:
cohort_final.head()

,PATIENT_ID,ENCOUNTER_ID,HOSPITAL_ENCOUNTER_DATE,AGE_AT_VISIT,DEATH_AT_VISIT_IND,COUNT_CURRENT_MEDS,CURRENT_OPIOID_IND,READMISSION_90_DAY_IND,READMISSION_30_DAY_IND,FIRST_READMISSION_DATE
421,708b81c9-21a9-411a-aae4-052ef8925b97,22874b3d-0873-40e4-a455-897d51d593f7,2012-02-18 21:50:51,25.309212,0,0.0,0,NaN,NaN,NaN
425,708b81c9-21a9-411a-aae4-052ef8925b97,134c5ee3-1b72-4e3e-a44a-3efebe77603b,2013-08-03 21:50:51,26.765779,0,0.0,0,NaN,NaN,NaN
429,708b81c9-21a9-411a-aae4-052ef8925b97,6125f147-72d4-48a4-9813-2a9dc878f055,2014-12-08 21:50:51,28.112829,0,0.0,0,NaN,NaN,NaN
438,708b81c9-21a9-411a-aae4-052ef8925b97,f837dcf8-af7d-43b8-b5c2-773d494617ab,2015-08-31 21:50:51,28.841112,0,0.0,0,NaN,NaN,NaN
4523,65b093e4-b353-447a-b9ae-290567f1c950,010594a6-a6ff-4875-9a9e-6b213367cde0,2018-12-10 00:59:35,23.020322,0,0.0,0,NaN,NaN,NaN


In [83]:
cohort_final.to_csv(r'C:\Users\katec\Documents\CHOP\analyst-take-home-task-master\analyst-take-home-task-master\datasets\cohort_final.csv')